<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [40]:
# Import libraries.
import pandas as pd
import spacy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.pipeline import Pipeline

In [2]:
# Load data.
df = pd.read_json('./data/review_sample.json', lines=True)

In [3]:
# Preview data.
df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


In [17]:
# Check data dimensions.
df.shape

(10000, 9)

In [18]:
# Display range and distribution of star ratings.
df['stars'].value_counts()

5    4462
4    2185
1    1496
3    1098
2     759
Name: stars, dtype: int64

## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [4]:
def tokenize(doc):
    """
    Accepts one document at a time; returns a list of tokens.
    """
    lemmas = []
    
    doc = nlp(doc)
    
    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False)) \
        and ((token.pos_!= 'PRON') and token.like_num == False):
            lemmas.append(token.lemma_)
    
    return lemmas

In [5]:
def clean(column):
    """
    Removes dollar signs and reduces any excess whitespace to a single
    space per instance.
    """
    column = column.copy()
    
    # Remove dollar signs.
    column = column.str.replace('$', '')
    
    # Reduce any excess whitespace to a single space per instance.
    column = column.str.replace(r'\s+', ' ')
    
    return column

In [6]:
nlp = spacy.load('en_core_web_lg')

In [7]:
# Preprocess and clean text.
df['text'] = clean(df['text'])

In [8]:
# Generate tokens from text.
tokens = df['text'].apply(tokenize)

In [9]:
# Preview tokens.
tokens[0]

['beware',
 'fake',
 'fake',
 'fake',
 'small',
 'business',
 'Los',
 'Alamitos',
 'receive',
 'look',
 'like',
 'legitimate',
 'bill',
 'account',
 'number',
 'call',
 'phone',
 'number',
 'list',
 'wait',
 'time',
 'hold',
 'say',
 'minute',
 'leave',
 'message',
 'live',
 'person',
 'phone',
 'matter',
 'number',
 'select',
 'leave',
 'firm',
 'message',
 'contact',
 'BBB',
 'attorney',
 'company',
 'try',
 'scam',
 'business',
 'illegal']

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

### Create a vector representation of the reviews

In [10]:
# Instantiate vectorizer object.
tfidf = TfidfVectorizer(min_df=0.01, 
                        max_df=0.98,
                        ngram_range=(1,2))

# Create a vocabulary and get word counts per document.
dtm = tfidf.fit_transform(tokens.astype(str))

# Get feature names to use as dataframe column headers.
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

# Preview feature matrix.
dtm.head()

,30,able,absolutely,accommodate,actually,add,affordable,afternoon,ago,agree,...,wrap,write,wrong,year,year ago,yelp,yes,young,yum,yummy
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.095275,0.0,0.0,0.0,0.146312,0.0,0.0
3,0.0,0.0,0.0,0.0,0.254845,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [11]:
# Instantiate nearest neighbors model.
nn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')

# Fit on TF-IDF Vectors
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

### Write a fake review and query for the 10 most similiar reviews

Print the text of the reviews. Do you notice any patterns?

In [12]:
# Create a fake negative review.
bad_review = ('Boy, the food at this place is really terrible, and '
              'such small portions.')

In [13]:
# Vectorize fake review.
bad_vec = tfidf.transform([bad_review])

In [14]:
# Find 10 most similar reviews.
bad_distances, bad_indices = nn.kneighbors(bad_vec.todense(), 10)

In [15]:
# Print star ratings and abbreviated text of similar reviews.
df.iloc[bad_indices[0]][['stars', 'text']]

,stars,text
2627,1,"Terrible terrible terrible went to get a ""hot ..."
2135,1,Terrible airline. They have nothing to offer a...
3230,1,This place just sucks. Food was terrible. Serv...
5298,1,Terrible.. absolutely terrible. Dr. Hinds is e...
7006,1,Yuk! Terrible pizza! I think I got food poison...
5366,1,Don't shop here. The owner John Menard is PURE...
4673,2,Terrible terrible terrible. After they lost my...
6204,5,旅行でラスベガスに来ましたがネイルがはげてるのが気になり、探したお店でした。 質問にも丁寧に...
6311,5,天氣很熱吃不下東西，今天我點了一個韓國冷面湯、餐後點了甜點，冰沙系列不會太甜膩，覺得店家很用...
1104,1,Fries were burnt to shit. Jalapeños were so bu...


In [16]:
# Print full text of similar reviews.
for text in df.iloc[bad_indices[0]]['text']:
    print(f'{text}\n')

Terrible terrible terrible went to get a "hot and ready pizza and box" which is supposed to be ready between the hours of 4 and 8 p.m. at 7:24 p.m. here it is 8 p.m. and I still have not received my order which I was told would be ready in 12 to 15 minutes this place is terrible terrible terrible and I will no longer be spending my money at this particular little Little Caesars if any Little Caesars Domino's here I come

Terrible airline. They have nothing to offer accept a ride. Why is it that airlines have terrible business models? Virgin America is my favorite. We need more Airlines like them.

This place just sucks. Food was terrible. Service was even worse some how. Just skip it all together.

Terrible.. absolutely terrible. Dr. Hinds is especially rude, very difficult to get anyone to return a phone call, just overall terrible patient care.

Yuk! Terrible pizza! I think I got food poisoning!! Yuk! Yuk! Yuk! Yuk! Yuk! Yuk! Yik! Still Yuk!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ Yikes yu

#### Observations

Unsurprisingly, our model does not handle non-English reviews well. With that exception noted, however, it otherwise does seem to do well at identifying clearly negative reviews.

On the other hand, the model does not seem to reliably pick up on the type of business being reviewed - we have an airline, a couple of medical providers, a hardware store, and several restaurant reviews all lumped together here.

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a pipeline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

### Create a Pipeline

Create a pipeline object with a sklearn CountVectorizer or TfidfVectorizer and any sklearn classifier. Use that pipeline to estimate a model to predict stars. Use the pipeline to predict a star rating for your fake review from Part 2.

In [21]:
# Create pipeline components.
vectorizer = TfidfVectorizer()
classifier = KNeighborsClassifier()

In [22]:
# Define the pipeline.
pipe = Pipeline([('vect', vectorizer), 
                 ('clf', classifier)])

In [34]:
# Fit pipeline to data.
pipe.fit(tokens.astype(str), df['stars'])

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,


In [37]:
# Use the pipeline to predict a star rating for a fake review.
pipe.predict([bad_review])

array([1], dtype=int64)

As expected, the pipeline predicts the lowest possible rating for this clearly negative review.

### Tune the Pipeline

Tune the entire pipeline with a GridSearch.

In [23]:
# Define parameter space for exploration.
parameters = {
    'vect__max_df': (0.95, 0.98),
    'vect__min_df': (0.02, 0.05),
    'vect__max_features': (500, 1000),
    'vect__ngram_range': ((1, 1), (1, 2)),
    'clf__n_neighbors': (5, 10),
    'clf__weights':('uniform', 'distance')
}

In [24]:
# Search parameter space for best model.
grid_search = GridSearchCV(pipe, parameters, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(tokens.astype(str), df['stars'])

Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   27.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 320 out of 320 | elapsed:  2.8min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                          

In [27]:
# Get accuracy of best model.
grid_search.best_score_

0.4892

In [42]:
# Compare to baseline accuracy.
accuracy_score([5]*len(df['stars']), df['stars'])

0.4462

That's pretty terrible! How wrong _are_ our mis-classified star ratings, though?

In [43]:
# Re-search grid, but penalize degree of error.
grid_search_mae = GridSearchCV(pipe, 
                               parameters, 
                               scoring='neg_mean_absolute_error', 
                               cv=3, 
                               n_jobs=-1, 
                               verbose=1)
grid_search_mae.fit(tokens.astype(str), df['stars'])

Fitting 3 folds for each of 64 candidates, totalling 192 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   24.8s
[Parallel(n_jobs=-1)]: Done 192 out of 192 | elapsed:  1.8min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                          

In [45]:
# Get MAE of best model.
abs(grid_search_mae.best_score_)

0.979

So on average, we're off by about one star. Not great, but not _terrible_ either.

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)